In [9]:
import nltk
nltk.download('punkt')

def count_sentences_arabic(text):
    sentences = nltk.sent_tokenize(text)
    return len(sentences)+1
def analyze_arabic_text(text):
    # Nombre de phrases dans le texte
    sentences = count_sentences_arabic(text)

    # Nombre de mots dans le texte
    words = len(text.split())

    # Nombre de caractères dans le texte
    characters = len(text)

    return sentences, words, characters

def analyze_arabic_file(file_path):
    # Lecture du contenu du fichier
    print(file_path)
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            file_content = file.read() # lire tout le contenu du fichier et stocker le dans la variable file_content

        # Analyse du texte du fichier en utilisant la fonction analyze_arabic_text
        sentences_count, words_count, characters_count = analyze_arabic_text(file_content)

        # Affichage des résultats
        #print("Le nombre de phrases dans le fichier est : ", sentences_count)
        #print("Le nombre de mots dans le fichier est : ", words_count)
        #print("Le nombre de caractères dans le fichier est : ", characters_count)
        return sentences_count, words_count, characters_count
        
    except FileNotFoundError:
        print(f"Fichier introuvable : {file_path}")
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\XPS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
from jpype import startJVM, shutdownJVM, java
import os
import sys
from pathlib import Path
import jpype
from jpype import *

def extraire_colonne_secondaire(texte):
    mots = texte.split('{{')
    colonne_secondaire = [mot.split(':')[1][:-2] if '}}' in mot else '' for mot in mots]
    colonne_secondaire = [mot.strip('}').strip() for mot in colonne_secondaire if mot]
    return colonne_secondaire


def lsp(chemin_fichier):
    #print(jpype.getDefaultJVMPath())
    path1='c:\\tmp'
    
    if not jpype.isJVMStarted():
        jpype.startJVM(jpype.getDefaultJVMPath() ,'-ea', classpath=[path1],convertStrings=True )
    
    path2= 'C:\\Users\\XPS\\Desktop\\FerhatOumaimaQassimiIkhlas\\PosTaggerAlKhalil.jar'
    jpype.addClassPath(path2)
    
    path3= 'C:\\Users\\XPS\\Desktop\\FerhatOumaimaQassimiIkhlas\\ADAT-Analyzer.jar'
    jpype.addClassPath(path3)
    
    #Import the Java class
    LemmaStemma = jpype.JClass("net.oujda_nlp_team.ADATAnalyzer")
    POStag = jpype.JClass("net.nlp.parser.TestParserClient")
    
    # Chemin vers le fichier
    #chemin_fichier = r"C:\Users\mimif\OneDrive\Bureau\S3\maz\Ressources lisibilité\niveaux\1\استئجار شقة.txt"
    #chemin_fichier = r"C:\Users\mimif\OneDrive\Bureau\S3\maz\Ressources lisibilité\niveaux\test.txt"
    
    with open(chemin_fichier, encoding="utf8") as file:
        file_content = file.read()
        
    #Lemmatizer, type de retour String
    lemmes = LemmaStemma.getInstance().processADATLemmatizerString(file_content)
    
    #Stemmer, type de retour String
    stems = LemmaStemma.getInstance().processADATStemmerString(file_content)

    #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    lemmes = extraire_colonne_secondaire(lemmes)
    stems = extraire_colonne_secondaire(stems)

     # Retirer les marqueurs "{{" et "}}" de chaque élément avant de les fusionner
    lemmes = [lemme.replace('{{', '').replace('}}', '') for lemme in lemmes]
    stems = [stem.replace('{{', '').replace('}}', '') for stem in stems]
    # Concaténer les lemmes et les stems en une seule chaîne de caractères
    #lemme_string = ' '.join(lemmes)
    #stem_string = ' '.join(stems)
    
    return  lemmes, stems#lemme_string, stem_string



In [11]:

from jpype import startJVM, shutdownJVM, java
import os
import sys
from pathlib import Path
import jpype
from jpype import *



def write_lemmas_to_file(lemmas_list):
    # Chemin vers le fichier temporaire
    file_path = r"C:\\Users\\XPS\\Desktop\\FerhatOumaimaQassimiIkhlas\\tmp.txt"  # Adapter le chemin si nécessaire
    
    # Écriture des lemmes dans le fichier temporaire
    with open(file_path, 'w', encoding='utf-8') as file:
        for lemma in lemmas_list:
            file.write(lemma + '\n')
    
    return file_path

def extraire_tags_POS(fichier_POS):
    # Lire le contenu du fichier de sortie POS
    with open(fichier_POS, 'r', encoding='utf-8') as file:
        contenu = file.read()
    
    # Extraire les tags PoS de chaque {{ }} dans le fichier de sortie POS
    tags_POS = []
    elements = contenu.split('{{')
    for element in elements:
        if '|' in element and '+' in element:
            tag = element.split('+')[1].split('|')[0]
            tags_POS.append(tag)

    # Construire une chaîne avec des espaces entre chaque tag POS
    tags_string = ' '.join(tags_POS)
    
    return tags_string

def POS(chemin_fichier):

    #chemin_fichier = write_lemmas_to_file(lems)
    #print(jpype.getDefaultJVMPath())
    path1='c:\\tmp'
    
    if not jpype.isJVMStarted():
        jpype.startJVM(jpype.getDefaultJVMPath() ,'-ea', classpath=[path1],convertStrings=True )

    path3= 'C:\\Users\\XPS\\Desktop\\FerhatOumaimaQassimiIkhlas\\ADAT-Analyzer.jar'
    jpype.addClassPath(path3)
    
    #Import the Java class
    POSclass = jpype.JClass("net.oujda_nlp_team.ADATAnalyzer")
    
    with open(chemin_fichier, encoding="utf8") as file:
        file_content = file.read()

    #POSTagger, type de retour .txt file
    POSclass.getInstance().processADATStemmerWithPOSFile(chemin_fichier, "utf-8", "output_POS.txt", "utf-8")
    
    #liste de pos tags
    chemin_fichier = r"C:\\Users\\XPS\\Desktop\\FerhatOumaimaQassimiIkhlas\\output_POS.txt"
    pos = extraire_tags_POS(chemin_fichier)
    
    return pos

def lemmes_ambigus(lems, pos):
    lemme_amb = []
    for lem in lems: #set(lems):  # Parcours des lemmes uniques
        indices = [i for i, x in enumerate(lems) if x == lem]  # Récupération des indices du lem dans lems
        pos_tmp = pos[indices[0]]  # Premier tag POS associé au lem
        
        for index in indices[1:]:  # Parcours des indices restants pour le même lem
            pos_actuel = pos[index]  # Tag POS actuel associé au lem
            if pos_actuel != pos_tmp:  # Comparaison des tags POS
                lemme_amb.append(lem)  # Si les tags POS sont différents, ajout à la liste des lemmes ambigus
                break  # Sortie de la boucle pour passer au lem suivant

    return lemme_amb


In [12]:
def enlever_ponctuation_arabe(fichier):
    # Liste des symboles de ponctuation arabes
    ponctuation_arabe = '،؍؛؟٪٫٬؞؟؛؛،۔؟٪٫٬٭٠١٢٣٤٥٦٧٨٩٪٪۔ۖۗۘۙۚۛۜ۝۞ۣ۟۠ۡۢۤۥۦۧۨ۩‘’“”•…:'
    
    # Lecture du fichier
    with open(fichier, 'r', encoding='utf-8') as file:
        contenu = file.read()

    # Suppression de la ponctuation arabe
    contenu_sans_ponctuation = ''.join(caractere for caractere in contenu if caractere not in ponctuation_arabe)

    # Écriture du contenu sans ponctuation dans le même fichier
    with open(fichier, 'w', encoding='utf-8') as file:
        file.write(contenu_sans_ponctuation)



In [13]:
import string

def enlever_ponctuation(fichier):
    # Liste des symboles de ponctuation
    ponctuation = string.punctuation

    # Lecture du fichier
    with open(fichier, 'r', encoding='utf-8') as file:
        contenu = file.read()

    # Suppression de la ponctuation
    '''When using join() in Python, the string specified before the join() method acts as a separator between the elements being joined.
     indicates that all the characters that pass the condition (i.e., 
     those not in ponctuation) will be concatenated directly without any spaces or separators between them, forming a single string.    
    '''
    contenu_sans_ponctuation = ''.join(caractere for caractere in contenu if caractere not in ponctuation) # '' emptycaract not space caract

    # Écriture du contenu sans ponctuation dans le même fichier
    with open(fichier, 'w', encoding='utf-8') as file:
        file.write(contenu_sans_ponctuation)



In [14]:
def count_element(lemmes, stems):
  
    # Calcul du nombre de lemmes générés
    nombre_lemmes = len(lemmes)

    # Calcul du nombre de stems
    nombre_stems = len(stems)
    
    # Création d'un ensemble (set) pour obtenir les lemmes distincts
    #Lorsque nous convertissons une liste (ou toute autre structure itérable) en un ensemble en utilisant set(), 
        #Python élimine automatiquement les doublons, ne laissant que les éléments uniques.
    lemmes_distincts = set(lemmes)
    
    # Calcul du nombre de lemmes distincts
    nombre_lemmes_distincts = len(lemmes_distincts)
    
    # Calcul du nombre de lemmes fréquents (apparaissant plus d'une fois)
    lemmes_freq = {}
    for lemme in lemmes:
        if lemme in lemmes_freq:
            lemmes_freq[lemme] += 1
        else:
            lemmes_freq[lemme] = 1

    nombre_lemmes_freq = sum(1 for lemme, freq in lemmes_freq.items() if freq > 1)
    
    return nombre_lemmes, nombre_lemmes_distincts, nombre_lemmes_freq, nombre_stems 



In [7]:
import os
import xlsxwriter

# Chemin du répertoire principal que vous souhaitez parcourir
chemin_repertoire = r"C:\Users\mimif\OneDrive\Bureau\S3\maz\Ressources lisibilité\niveaux"

# Chemin du fichier xlsx où vous souhaitez stocker les données
chemin_fichier = r"C:\\Users\\XPS\\Desktop\\FerhatOumaimaQassimiIkhlas\\res.xlsx"
workbook = xlsxwriter.Workbook(chemin_fichier)

# The workbook object is then used to add new 
# worksheet via the add_worksheet() method.
worksheet = workbook.add_worksheet()

# Écriture des en-têtes
worksheet.write('A1', 'Nom du fichier')
worksheet.write('B1', 'Contenu')
worksheet.write('C1', 'sentences_count')
worksheet.write('D1', 'words_count')
worksheet.write('E1', 'characters_count')
worksheet.write('F1', 'lemmes')
worksheet.write('G1', 'lemmes_ambigus')
worksheet.write('H1', 'nombre_lemmes')
worksheet.write('I1', 'nombre_lemmes_distincts')
worksheet.write('J1', 'nombre_lemmes_ambigus')
worksheet.write('K1', 'nombre_lemmes_freq')
worksheet.write('L1', 'stems')
worksheet.write('M1', 'nombre_stems')
worksheet.write('N1', 'POS')
worksheet.write('O1', 'classe')
ligne = 1  # Pour commencer à écrire à partir de la deuxième ligne (après les en-têtes)
# Parcours récursif des fichiers et sous-répertoires
for repertoire_actuel, sous_repertoires, fichiers in os.walk(chemin_repertoire):
    # Pour chaque fichier trouvé dans le répertoire actuel
    for nom_fichier in fichiers:
        # Afficher le nom du sous-répertoire et le nom du fichier
        #os.path.basename(repertoire_actuel) pour avoir seulement le nom du sous rep sans le chemin absolu 
        #print(f"Sous-répertoire : {os.path.basename(repertoire_actuel)}, Fichier : {nom_fichier}")
        classe = os.path.basename(repertoire_actuel)
        path = repertoire_actuel + "\\" + nom_fichier

        # Calcul nbr de phrases, mots et caracts
        sentences_count, words_count, characters_count = analyze_arabic_file(path)
        # Enlever la ponctuation
        enlever_ponctuation_arabe(path)
        enlever_ponctuation(path)
        # Extraction de lemmes et de stems
        lemmes, stems = lsp(path)
        #liste de posTagger
        write_lemmas_to_file(lemmes)
        file_path = r"C:\Users\mimif\OneDrive\Bureau\S3\maz\Ressources lisibilité\tmp.txt"
        pos = POS(file_path)
        # calcul nombre de lemme, lemmeDinstinct, lemmeFreq, et nombre de stem
        nombre_lemmes, nombre_lemmes_distincts, nombre_lemmes_freq, nombre_stems = count_element(lemmes, stems)
        #les lemmes ambigus
        lem_amb = lemmes_ambigus(lemmes, pos)
        #le nombre de lemmes ambigus
        nombre_lemmes_ambigus = len(lem_amb)
        # Construire une chaîne avec des espaces entre chaque tag POS
        lemmes = ' '.join(lemmes)
        lem_amb = ' '.join(lem_amb)
        stems = ' '.join(stems)
        
        #print(path)
        with open(path, encoding="utf8") as file:
            contenu = file.read()
        # Écrire les données dans le fichier Excel
        worksheet.write(ligne, 0, nom_fichier)  # Colonne A
        worksheet.write(ligne, 1, contenu)
        worksheet.write(ligne, 2, sentences_count)
        worksheet.write(ligne, 3, words_count)
        worksheet.write(ligne, 4, characters_count)
        worksheet.write(ligne, 5, lemmes)
        worksheet.write(ligne, 6, lem_amb)
        worksheet.write(ligne, 7, nombre_lemmes)
        worksheet.write(ligne, 8, nombre_lemmes_ambigus)
        worksheet.write(ligne, 9, nombre_lemmes_distincts)
        worksheet.write(ligne, 10, nombre_lemmes_freq)
        worksheet.write(ligne, 11, stems)
        worksheet.write(ligne, 12, nombre_stems)  
        worksheet.write(ligne, 13, pos) 
        worksheet.write(ligne, 14, classe)  # Colonne L
        ligne += 1  # Passage à la ligne suivante
        
# Fermeture du classeur
workbook.close()

In [21]:
import pandas as pd

# Chemin du fichier à traiter
chemin_fichier = r"C:\\Users\\XPS\\Desktop\\FerhatOumaimaQassimiIkhlas\\fichier.txt"

# Calcul nbr de phrases, mots et caracts
sentences_count, words_count, characters_count = analyze_arabic_file(chemin_fichier)
# Enlever la ponctuation
enlever_ponctuation_arabe(chemin_fichier)
enlever_ponctuation(chemin_fichier)
# Extraction de lemmes et de stems
lemmes, stems = lsp(chemin_fichier)
#liste de posTagger
write_lemmas_to_file(lemmes)
file_path = r"C:\\Users\\XPS\\Desktop\\FerhatOumaimaQassimiIkhlas\\tmp.txt"
pos = POS(file_path)
# calcul nombre de lemme, lemmeDinstinct, lemmeFreq, et nombre de stem
nombre_lemmes, nombre_lemmes_distincts, nombre_lemmes_freq, nombre_stems = count_element(lemmes, stems)
#les lemmes ambigus
lem_amb = lemmes_ambigus(lemmes, pos)
#le nombre de lemmes ambigus
nombre_lemmes_ambigus = len(lem_amb)
# Construire une chaîne avec des espaces entre chaque tag POS
lemmes = ' '.join(lemmes)
lem_amb = ' '.join(lem_amb)
stems = ' '.join(stems)

with open(chemin_fichier, encoding="utf8") as file:
    contenu = file.read()

# Création d'un DataFrame avec les données
df = pd.DataFrame({
    'Nom du fichier': [os.path.basename(chemin_fichier)],
    'Contenu': [contenu],
    'SL1': [sentences_count],
    'WL1': [words_count],
    'WL5': [characters_count],
    'lemmes': [lemmes],
    'lemmes_ambigus': [lem_amb],
    'nombre_lemmes': [nombre_lemmes],
    'VL1': [nombre_lemmes_distincts],
    'AMb1': [nombre_lemmes_ambigus],
    'VL2': [nombre_lemmes_freq],
    'stems': [stems],
    'WL4': [nombre_stems],
    'POS': [pos],
    'classe': 'inconu'  # Vous devez spécifier la classe pour le fichier
})

# Enregistrement du DataFrame dans un fichier CSV
chemin_csv = r"C:\\Users\\XPS\\Desktop\\FerhatOumaimaQassimiIkhlas\\fichier.csv"
df.to_csv(chemin_csv, index=False)  # Ne pas inclure l'index dans le fichier CSV


C:\\Users\\XPS\\Desktop\\FerhatOumaimaQassimiIkhlas\\fichier.txt


In [22]:
import os
import pandas as pd

def process_and_save_file(chemin_fichier):
    # Calcul nbr de phrases, mots et caracts
    sentences_count, words_count, characters_count = analyze_arabic_file(chemin_fichier)
    # Enlever la ponctuation
    enlever_ponctuation_arabe(chemin_fichier)
    enlever_ponctuation(chemin_fichier)
    # Extraction de lemmes et de stems
    lemmes, stems = lsp(chemin_fichier)
    #liste de posTagger
    write_lemmas_to_file(lemmes)
    file_path = r"C:\\Users\\XPS\\Desktop\\FerhatOumaimaQassimiIkhlas\\tmp.txt"
    pos = POS(file_path)
    # calcul nombre de lemme, lemmeDinstinct, lemmeFreq, et nombre de stem
    nombre_lemmes, nombre_lemmes_distincts, nombre_lemmes_freq, nombre_stems = count_element(lemmes, stems)
    #les lemmes ambigus
    lem_amb = lemmes_ambigus(lemmes, pos)
    #le nombre de lemmes ambigus
    nombre_lemmes_ambigus = len(lem_amb)
    # Construire une chaîne avec des espaces entre chaque tag POS
    lemmes = ' '.join(lemmes)
    lem_amb = ' '.join(lem_amb)
    stems = ' '.join(stems)

    with open(chemin_fichier, encoding="utf8") as file:
        contenu = file.read()

    # Création d'un DataFrame avec les données
    df = pd.DataFrame({
        'Nom du fichier': [os.path.basename(chemin_fichier)],
        'Contenu': [contenu],
        'SL1': [sentences_count],
        'WL1': [words_count],
        'WL5': [characters_count],
        'lemmes': [lemmes],
        'lemmes_ambigus': [lem_amb],
        'nombre_lemmes': [nombre_lemmes],
        'VL1': [nombre_lemmes_distincts],
        'AMb1': [nombre_lemmes_ambigus],
        'VL2': [nombre_lemmes_freq],
        'stems': [stems],
        'WL4': [nombre_stems],
        'POS': [pos],
        'classe': 'inconnu'  # Spécifiez la classe pour le fichier
    })

    # Enregistrement du DataFrame dans un fichier CSV
    chemin_csv = r"C:\\Users\\XPS\\Desktop\\FerhatOumaimaQassimiIkhlas\\fichier.csv"
    df.to_csv(chemin_csv, index=False)  # Ne pas inclure l'index dans le fichier CSV



In [23]:
# Appel de la fonction avec le chemin du fichier à traiter
chemin_fichier = r"C:\\Users\\XPS\\Desktop\\FerhatOumaimaQassimiIkhlas\\fichier.txt"
process_and_save_file(chemin_fichier)


C:\\Users\\XPS\\Desktop\\FerhatOumaimaQassimiIkhlas\\fichier.txt
